In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble

from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
IBMQ.load_accounts()

# Load simulator
local_simulator = Aer.get_backend('qasm_simulator')

# Introduction

In [ ]:
qr = QuantumRegister(2, name="q")
cr = ClassicalRegister(1, name="result")
circuit = QuantumCircuit(qr, cr)
circuit.x(qr)
circuit.measure(qr[0], cr)
circuit.draw()

In [ ]:
job = execute(circuit, backend=local_simulator)
job.result().get_counts()

# Bloch sphere

In [ ]:
from qiskit import BasicAer
backend_sv_sim = BasicAer.get_backend('statevector_simulator')
qr = QuantumRegister(1)
circ = QuantumCircuit(qr)
#circ.x(qr)
#circ.z(qr)
#circ.y(qr)
result = execute(circ, backend_sv_sim).result()
plot_bloch_multivector(result.get_statevector())

In [ ]:
from qiskit import BasicAer
backend_sv_sim = BasicAer.get_backend('statevector_simulator')
qr = QuantumRegister(3)
cr = ClassicalRegister(3)
circ = QuantumCircuit(qr, cr)
circ.h(qr[0])
circ.h(qr[1])
circ.h(qr[2])
# circ.measure(qr, cr)
result = execute(circ, backend_sv_sim).result()
for i in result.get_statevector():
    print(i)

# Deutsch

## Oracle 1: Constant zero

In [ ]:
input = QuantumRegister(1, name='input')
temp = QuantumRegister(1, name='temp')
constant0 = QuantumCircuit(input, temp, name='oracle')
oracle = constant0.to_instruction()
constant0.draw()

## Oracle 2: Identity

In [ ]:
input = QuantumRegister(1, name='input')
temp = QuantumRegister(1, name='temp')
identity = QuantumCircuit(input, temp, name='oracle')
identity.cx(input, temp)
oracle = identity.to_instruction()
identity.draw()

## Oracle 3: Invert

In [ ]:
input = QuantumRegister(1, name='input')
temp = QuantumRegister(1, name='temp')
invert = QuantumCircuit(input, temp, name='oracle')
invert.cx(input, temp)
invert.x(temp)
oracle = identity.to_instruction()
invert.draw()

## Oracle 4: Constant one

In [ ]:
input = QuantumRegister(1, name='input')
temp = QuantumRegister(1, name='temp')
constant1 = QuantumCircuit(input, temp, name='oracle')
constant1.x(temp)
oracle = constant1.to_instruction()
constant1.draw()

## Run an oracle

In [ ]:
input = QuantumRegister(1, name='input')
temp = QuantumRegister(1, name='temp')
output = ClassicalRegister(1, name='output')

circuit = QuantumCircuit(input, temp, output)
circuit.x(input) # <- set input to 1
circuit.barrier()
circuit += identity
circuit.barrier()
circuit.measure(temp, output)
circuit.draw()

In [ ]:
execute(circuit, backend=local_simulator).result().get_counts()

## Running Deutsch's Algorithm

In [ ]:
qr = QuantumRegister(2)
cr = ClassicalRegister(1)
circuit = QuantumCircuit(qr, cr)
circuit.x(qr[1]);
circuit.h(qr)
circuit.append(oracle, [qr[0], qr[1]])
circuit.h(qr[0])
circuit.measure(qr[0], cr[0]);
circuit.draw()

In [ ]:
counts = execute(circuit, backend=local_simulator).result().get_counts()
if '1' in counts:
    print('BALANCED')
elif '0' in counts:
    print('CONSTANT')
print(counts)

# The real device!

In [ ]:
from qiskit.providers.ibmq import least_busy
least_busy_device = least_busy(IBMQ.backends(simulator=False))

In [ ]:
counts = execute(circuit, backend=least_busy_device).result().get_counts()
print(counts)

In [ ]:
%qiskit_backend_overview

In [ ]:
ibmqx2 = IBMQ.get_backend('ibmqx2')
%qiskit_backend_monitor ibmqx2